In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [3]:
data=pd.read_csv('final_data_en.csv')
data=data[['text','class']]
data

,text,class
0,RT @maideeeni: I want to say a very big thank ...,anxiety
1,@AisYalcaE @clyrhck Done! I just need 320! Tha...,lonely
2,@SAD_istfied I'm gonna cry /pos,stress
3,RT @JoshOfficial___: I need a consistent fuck ...,anxiety
4,RT @mikegolicjr: glad we're finally talking ab...,normal
...,...,...
39761,Im nervous ash to fly though,anxiety
39762,64E7B6DA :Battle ID\nI need backup!\nLvl 120 G...,lonely
39763,SNOW MUCH FUN writing about our snow days in @...,anxiety
39764,Very time sensitive. I'm trans and my roommate...,anxiety


In [4]:
import numpy as np
import tensorflow as tf
# train=data.loc[ :33802]
# test=data.loc[33803: ]
data=data.sample(frac=1)


data

,text,class
13264,RT @o_keilani: I like my men in love with me w...,lonely
27080,"@Foxfire40900590 Marked by Alita's character, ...",stress
4924,"@Alienorexium Right, ""MCU style"". I was so dis...",normal
22240,RT @NeverGi19090195: @Dolphin_Project A pod of...,stress
559,"The help I need from you is, I got the pictur...",anxiety
...,...,...
1275,They'd better buckle up. The next song starts ...,stress
105,RT @xxCatastrophee: Should I post a video like...,lonely
15948,RT @EricMMatheny: As a criminal defense attorn...,lonely
33918,"@ALIENQUEENTOME maybe,,,, idk i just yell out ...",anxiety


In [5]:
data.dropna(inplace=True)



In [6]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [7]:
data

,text,class
13264,RT @o_keilani: I like my men in love with me w...,lonely
27080,"@Foxfire40900590 Marked by Alita's character, ...",stress
4924,"@Alienorexium Right, ""MCU style"". I was so dis...",normal
22240,RT @NeverGi19090195: @Dolphin_Project A pod of...,stress
559,"The help I need from you is, I got the pictur...",anxiety
...,...,...
1275,They'd better buckle up. The next song starts ...,stress
105,RT @xxCatastrophee: Should I post a video like...,lonely
15948,RT @EricMMatheny: As a criminal defense attorn...,lonely
33918,"@ALIENQUEENTOME maybe,,,, idk i just yell out ...",anxiety


In [8]:
import pandas as pd
import sklearn
import numpy as np
import re
import nltk

import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string

In [9]:
cachedStopWords = list(esw)
lemmatizer=WordNetLemmatizer()
#ps = PorterStemmer()

def lemmatize_article(sentence):
    sentence = word_tokenize(sentence)
    res = ''
    for word, tag in pos_tag(sentence):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        word = lemmatizer.lemmatize(word, wntag) if wntag else word
        res += word + ' '
    return res


def remove_stop_words(sentence):
    return ' '.join([word for word in sentence.split() if word not in cachedStopWords])
    
def remove_short(sentence):
    return ' '.join([word for word in sentence.split() if len(word) >= 3])
    
def remove_digits(sentence):
    return ' '.join([i for i in sentence.split() if not i.isdigit()])
    
def preprocess(all_texts):
    all_texts = list(map(lambda x: x.lower(), all_texts))
    all_texts = list(map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), all_texts))
    all_texts = list(map(lambda x: lemmatize_article(x), all_texts))
    all_texts = list(map(lambda x: x.strip(), all_texts))
    all_texts = list(map(lambda x: remove_stop_words(x), all_texts))
    all_texts = list(map(lambda x: remove_short(x), all_texts))
    all_texts = list(map(lambda x: remove_digits(x), all_texts))
    return all_texts

In [10]:
data['text'] = data['text'].str.replace(r'\s*@\w+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*\B@\w+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*@\S+', '', regex = True)
data['text'] = data['text'].str.replace(r'\s*@\S+\b', '', regex = True)
data['text'] = data['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data


,text,class
13264,RT: I like my men in love with me when they me...,lonely
27080,"Marked by Alita's character, A little sad be...",stress
4924,"Right, ""MCU style"". I was so disgusted by wha...",normal
22240,RT: A pod of striped dolphins were victimized!...,stress
559,"The help I need from you is, I got the pictur...",anxiety
...,...,...
1275,They'd better buckle up. The next song starts ...,stress
105,RT: Should I post a video like this (but with ...,lonely
15948,"RT: As a criminal defense attorney, I think I ...",lonely
33918,"maybe,,,, idk i just yell out ur name wheneve...",anxiety


In [11]:
data['text'] = preprocess(data['text'])
data

,text,class
13264,like men love meet really love know,lonely
27080,mark alitas character little sad hugos death m...,stress
4924,right mcu style disgust screen point just roll,normal
22240,pod striped dolphin victimize entire pod inclu...,stress
559,help need picture photographer picture just lo...,anxiety
...,...,...
1275,theyd buckle song start hate undergroundyou te...,stress
105,post video like panty onlyfans need follower t...,lonely
15948,criminal defense attorney think need expand pr...,lonely
33918,maybe idk just yell panicking need help,anxiety


In [12]:
data = pd.DataFrame(data)
data.head()

,text,class
13264,like men love meet really love know,lonely
27080,mark alitas character little sad hugos death m...,stress
4924,right mcu style disgust screen point just roll,normal
22240,pod striped dolphin victimize entire pod inclu...,stress
559,help need picture photographer picture just lo...,anxiety


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier

In [14]:
data = data[(data["text"] != "") & (data["text"] != "null")]


In [15]:
data.shape

(39555, 2)

In [16]:
data['class'].shape

(39555,)

In [31]:
X = data['text'].values
Y = data['class'].values


(39555,)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X).toarray()

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,
                                                    random_state = 0)

In [34]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

# Classification metrics
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.6441392095727648

Classification Report

               precision    recall  f1-score   support

     anxiety       0.60      0.70      0.65      3783
      lonely       0.59      0.49      0.54      2579
      normal       0.58      0.57      0.58      2915
      stress       0.84      0.80      0.82      2590

    accuracy                           0.64     11867
   macro avg       0.65      0.64      0.64     11867
weighted avg       0.65      0.64      0.64     11867



In [35]:
import re
classifiers = [
    LogisticRegression(solver="sag", random_state=42),
    LinearSVC(random_state=42),
    RandomForestClassifier(random_state=1),
    XGBClassifier(random_state=42),
    MLPClassifier(
        random_state=42,
        solver="adam",
        hidden_layer_sizes=(500, 500, 400),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),
]
# get names of the objects in list (too lazy for c&p...)
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier']


In [36]:
%%time
import pickle
import sklearn

# test all classifiers and save pred. results on test data
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    report = sklearn.metrics.classification_report(y_test, prediction)
    results[name] = report

Training classifier: LogisticRegression
Training classifier: LinearSVC
Training classifier: RandomForestClassifier
Training classifier: XGBClassifier
Training classifier: MLPClassifier
CPU times: user 21min 59s, sys: 55.6 s, total: 22min 54s
Wall time: 21min 12s


In [37]:
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

Results for LogisticRegression:
              precision    recall  f1-score   support

     anxiety       0.76      0.69      0.72      3783
      lonely       0.63      0.62      0.62      2579
      normal       0.76      0.90      0.82      2915
      stress       0.96      0.92      0.94      2590

    accuracy                           0.77     11867
   macro avg       0.78      0.78      0.78     11867
weighted avg       0.77      0.77      0.77     11867


Results for LinearSVC:
              precision    recall  f1-score   support

     anxiety       0.74      0.69      0.71      3783
      lonely       0.60      0.64      0.62      2579
      normal       0.80      0.84      0.82      2915
      stress       0.94      0.94      0.94      2590

    accuracy                           0.77     11867
   macro avg       0.77      0.78      0.77     11867
weighted avg       0.77      0.77      0.77     11867


Results for RandomForestClassifier:
              precision    recall  f1